In [1]:
import pandas as pd
import os
from src.python.classes.CallsClass import CallsClass
import requests
from pprint import pprint
from tqdm import tqdm
from time import sleep
from IPython.display import clear_output
from tabulate import tabulate
import inspect

In [2]:
callings = CallsClass()
df = callings.load_and_parse_forms()
df_teste = df.head(5)

Arquivos encontrados:
 ['forms_results_desafio_da_ponte.xlsx']

Arquivo carregado. Parseando...
+------------------------------+------------------+
|                              | qtde (percent)   |
|------------------------------+------------------|
| total de ceps                | 4539 (100.00%)   |
| ceps com underline           | 20 (0.44%)       |
| ceps sem hifen               | 18 (0.40%)       |
| ceps ok                      | 4539 (100.00%)   |
| ceps com menos de 8 dígitos  | 0 (0.00%)        |
| ceps com mais de 8 dígitos   | 0 (0.00%)        |
| ceps com tratamento especial | 0 (0.00%)        |
+------------------------------+------------------+
Parseado com sucesso.


# aqui

In [3]:
gets_dict = callings.brasilapi_one(df_teste)

100%|██████████| 5/5 [00:09<00:00,  2.09s/it]

num_iteration: 4
+------+-------+
|   ok |   nok |
|------+-------|
|    5 |     0 |
+------+-------+


100%|██████████| 5/5 [00:11<00:00,  2.28s/it]


In [4]:
gets_dict.get('brasilapi1_df')
# gets_dict.get('brasilapi1_df_logs')

,cep,state,city,neighborhood,street,service
0,22221090,RJ,Rio de Janeiro,Laranjeiras,Rua Paulo César de Andrade,open-cep
1,22030010,RJ,Rio de Janeiro,Copacabana,Rua Assis Brasil,open-cep
2,24220070,RJ,Niterói,Icaraí,Rua Lopes Trovão,open-cep
3,22610095,RJ,Rio de Janeiro,São Conrado,Avenida Prefeito Mendes de Morais,open-cep
4,20745312,RJ,Rio de Janeiro,Água Santa,Rua Dois de Fevereiro,open-cep


In [ ]:
cep = '2-279352'


In [44]:
cep = '22793-520'
get_cep = requests.get(f'https://brasilapi.com.br/api/cep/v2/{cep}').json()
get_cep

{'cep': '22793520',
 'state': 'RJ',
 'city': 'Rio de Janeiro',
 'neighborhood': 'Barra da Tijuca',
 'street': 'Rua Ataulpho Coutinho',
 'service': 'open-cep',
 'location': {'type': 'Point',
  'coordinates': {'longitude': '-43.3773358', 'latitude': '-23.0040166'}}}

In [79]:
# brasilapi v1


df_test = pd.DataFrame({
    'parsed_ceps': ['2279-3520', '2-279352', '2279-3520']
})

for idx, row in df_test.iterrows():
    df_test['parsed_ceps'].iloc[idx] = row['parsed_ceps'].replace('-', '')
    print(row.iloc[0])
df_test

2279-3520
2-279352
2279-3520


# aqui

In [35]:
import pandas as pd
from time import sleep
import requests
from pprint import pprint

url = 'https://brasilapi.com.br/api/cep/v1/{cep}'
df_test = pd.DataFrame({
    'parsed_ceps': ['2279-3520', '2-279352', '2279-3520']
})

for idx, row in df_test.iterrows():
    parsed_url = url.replace('{cep}', row['parsed_ceps'])
    get_cep = requests.get(parsed_url).json()
    if get_cep.get('errors') == None:
        pprint(get_cep, '\n')
    else:
        print('DFGDGHFSGFHGFHDSGFHGDFHDGHD\n')
    sleep(2)



parsed_ceps    2279-3520
Name: 0, dtype: object
{'cep': '22793520',
 'city': 'Rio de Janeiro',
 'neighborhood': 'Barra da Tijuca',
 'service': 'open-cep',
 'state': 'RJ',
 'street': 'Rua Ataulpho Coutinho'}
DFGDGHFSGFHGFHDSGFHGDFHDGHD
parsed_ceps    2279-3520
Name: 2, dtype: object
{'cep': '22793520',
 'city': 'Rio de Janeiro',
 'neighborhood': 'Barra da Tijuca',
 'service': 'open-cep',
 'state': 'RJ',
 'street': 'Rua Ataulpho Coutinho'}


In [95]:
# brasilapi v1
brasilapi1_df = pd.DataFrame({
    'brasilapi_cep': [],
    'brasilapi_state': [],
    'brasilapi_city': [],
    'brasilapi_neighborhood': [],
    'brasilapi_street': [],
    'brasilapi_service': [],
})
brasilapi1_df_logs = pd.DataFrame({
    'ok': [0],
    'nok': [0]
})

df_test = pd.DataFrame({
    'parsed_ceps': ['22793520', '2279352', '22793520']
})
with tqdm(total=len(df_test)) as pbar:
    for idx, row in df_test.head(5).iterrows():
        
        clear_output(wait=True)
        replace_cep = row['parsed_ceps']
        get_cep = requests.get(f'https://brasilapi.com.br/api/cep/v1/{replace_cep}').json()
        
        if get_cep.get('cep'):
            print(row)
            brasilapi1_df.loc[idx,'brasilapi_cep'] = get_cep.get('cep')
            brasilapi1_df.loc[idx,'brasilapi_state'] = get_cep.get('state')
            brasilapi1_df.loc[idx,'brasilapi_city'] = get_cep.get('city')
            brasilapi1_df.loc[idx,'brasilapi_neighborhood'] = get_cep.get('neighborhood')
            brasilapi1_df.loc[idx,'brasilapi_street'] = get_cep.get('street')
            brasilapi1_df.loc[idx,'brasilapi_service'] = get_cep.get('service')
            brasilapi1_df_logs.loc[0,'ok'] += 1
        else:
            brasilapi1_df_logs.loc[0,'nok'] += 1
        
        print(tabulate(brasilapi1_df_logs, headers='keys', tablefmt='psql', showindex=False))
        pbar.update()
        sleep(2)
df_test

100%|██████████| 3/3 [00:05<00:00,  1.99s/it]

parsed_ceps    22793520
Name: 2, dtype: object
+--------------------+---------------------+
|   brasilapi1_df_ok |   brasilapi1_df_nok |
|--------------------+---------------------|
|                  2 |                   1 |
+--------------------+---------------------+


100%|██████████| 3/3 [00:07<00:00,  2.46s/it]


,parsed_ceps
0,22793520
1,2279352
2,22793520


In [96]:
brasilapi1_df

,brasilapi_cep,brasilapi_state,brasilapi_city,brasilapi_neighborhood,brasilapi_street,brasilapi_service
0,22793520,RJ,Rio de Janeiro,Barra da Tijuca,Rua Ataulpho Coutinho,open-cep
2,22793520,RJ,Rio de Janeiro,Barra da Tijuca,Rua Ataulpho Coutinho,open-cep


In [35]:
# brasilapi v2
df_test = pd.DataFrame({
    'parsed_ceps': ['22793520', '2279352', '22793520']
})
for idx, row in df_test.head(5).iterrows():
    replace_cep = row['parsed_ceps']
    print(f'https://viacep.com.br/ws/{replace_cep}/json/')
    get_cep = requests.get(f'https://brasilapi.com.br/api/cep/v2/{replace_cep}').json()
    pprint(get_cep)

{'errors': [{'message': 'CEP informado possui mais do que 8 caracteres.',
             'service': 'cep_validation'}],
 'message': 'CEP deve conter exatamente 8 caracteres.',
 'name': 'CepPromiseError',
 'type': 'validation_error'}


In [43]:
# viacep
df_test = pd.DataFrame({
    'parsed_ceps': ['22793520', '2279352', '22793520']
})
try:
    for idx, row in df_test.head(5).iterrows():
        replace_cep = row['parsed_ceps']
        print(f'https://viacep.com.br/ws/{replace_cep}/json/')
        get_cep = requests.get(f'https://viacep.com.br/ws/{replace_cep}/json/').json()
        pprint(get_cep)
except Exception as e:
    print(e.args[0])

https://viacep.com.br/ws/22793520/json/
{'bairro': 'Barra da Tijuca',
 'cep': '22793-520',
 'complemento': '',
 'ddd': '21',
 'estado': 'Rio de Janeiro',
 'gia': '',
 'ibge': '3304557',
 'localidade': 'Rio de Janeiro',
 'logradouro': 'Rua Ataulpho Coutinho',
 'regiao': 'Sudeste',
 'siafi': '6001',
 'uf': 'RJ',
 'unidade': ''}
https://viacep.com.br/ws/22793520/json/
{'bairro': 'Barra da Tijuca',
 'cep': '22793-520',
 'complemento': '',
 'ddd': '21',
 'estado': 'Rio de Janeiro',
 'gia': '',
 'ibge': '3304557',
 'localidade': 'Rio de Janeiro',
 'logradouro': 'Rua Ataulpho Coutinho',
 'regiao': 'Sudeste',
 'siafi': '6001',
 'uf': 'RJ',
 'unidade': ''}


In [41]:
# apicep
df_test = pd.DataFrame({
    'parsed_ceps': ['22793520', '2279352', '22793520']
})
for idx, row in df_test.head(5).iterrows():
    replace_cep = row['parsed_ceps']
    get_cep = requests.get(f'https://cdn.apicep.com/file/apicep/{replace_cep}.json').json()
    pprint(get_cep)

{'code': 'not_found',
 'message': 'File with such name does not exist.',
 'status': 404}


**brasilapi v1** -> https://brasilapi.com.br/api/cep/v1/2279352
```json
{'errors': [{'message': 'CEP informado possui mais do que 8 caracteres.',
             'service': 'cep_validation'}],
 'message': 'CEP deve conter exatamente 8 caracteres.',
 'name': 'CepPromiseError',
 'type': 'validation_error'}
```

**brasilapi v2** -> 'https://brasilapi.com.br/api/cep/v2/2279352'
```json
{'errors': [{'message': 'CEP informado possui mais do que 8 caracteres.',
             'service': 'cep_validation'}],
 'message': 'CEP deve conter exatamente 8 caracteres.',
 'name': 'CepPromiseError',
 'type': 'validation_error'}
```

**viacep** -> 'https://viacep.com.br/ws/2279352/json/'
```error
JSONDecodeError: Expecting value: line 1 column 1 (char 0)
```

**apicep** -> 'https://cdn.apicep.com/file/apicep/2279352.json'
```json
{'code': 'not_found',
 'message': 'File with such name does not exist.',
 'status': 404}
```